<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import argparse
from pathlib import Path
import os

from veloce_luminosa_reduction import config

from veloce_luminosa_reduction.plotting import plot_ccd_imshow, create_rainbow_colormap, create_transparent_greyscale_colormap
from veloce_luminosa_reduction.utils import identify_calibration_and_science_runs, match_month_to_date, read_veloce_fits_image_and_metadata, polynomial_function, radial_velocity_shift, interpolate_spectrum
from veloce_luminosa_reduction.reduction import substract_overscan, extract_initial_order_ranges_and_coeffs
from veloce_luminosa_reduction.calibration import get_wavelength_coeffs_from_vdarc
from veloce_luminosa_reduction.korg_synthesis import calculate_synthetic_korg_spectrum
from veloce_luminosa_reduction.post_processing import degrade_resolution_with_uncertainty, interpolate_orders_and_merge, coadd_spectra

import time
import numpy as np
from numpy.polynomial.chebyshev import Chebyshev
from scipy.signal import medfilt, find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u
SSO = EarthLocation.of_site('Siding Spring Observatory')

In [2]:
config.date = '240219'

In [3]:
# Extract relevant information from night log
calibration_runs, science_runs = identify_calibration_and_science_runs(config.date, config.working_directory+'raw_data/')



Identifying calibration and science runs now

More than 1 Log file present, continuing with /Users/buder/git/veloce_luminosa_reduction/raw_data//240219/240219-AAT-2023B-24.log

Warning for HIP71683 (run 0140): Rosso slightly saturated.
Warning for HIP71683 (run 0140): Rosso slightly saturated.
Warning for HIP71683 (run 0140): Rosso slightly saturated.


In [4]:
# Extract tramlines
initial_order_ranges, initial_order_coeffs = extract_initial_order_ranges_and_coeffs()

In [12]:
def extract_orders(ccd1_runs, ccd2_runs, ccd3_runs, Flat = False, LC = False, tramline_debug = False):
    
    # Extract Images from CCDs 1-3
    images = dict()
    
    # Read in, overscan subtract and append images to array
    for ccd in [1,2,3]:
        
        images['ccd_'+str(ccd)] = []
        if ccd == 1: runs = ccd1_runs
        if ccd == 2: runs = ccd2_runs
        if ccd == 3: runs = ccd3_runs
        
        for run in runs:
            full_image, metadata = read_veloce_fits_image_and_metadata(config.working_directory+'raw_data/'+config.date+'/ccd_'+str(ccd)+'/'+config.date[-2:]+match_month_to_date(config.date)+str(ccd)+run+'.fits')
            trimmed_image, os_median, os_rms = substract_overscan(full_image, metadata)
            images['ccd_'+str(ccd)].append(trimmed_image)
        
        # Normalise across runs.
        images['ccd_'+str(ccd)] = np.array(np.median(images['ccd_'+str(ccd)],axis=0),dtype=float)
        if Flat:
            images['ccd_'+str(ccd)] /= np.nanmax(images['ccd_'+str(ccd)])

    counts_in_orders = []
    
    if tramline_debug:
        plt.figure(figsize=(15,15))
        s = plt.imshow(images['ccd_2'], vmin = 1, vmax = 20, cmap='Greys',aspect=5)
        plt.colorbar(s)
    
    for order in initial_order_coeffs:
        ccd = order[4]

        # Identify the tramline ranges for each order
        # initial_order_ranges[order] are the initial orders reported by C.Tinney.
        left = -45
        right = 0
        if LC & (ccd == '3'):
            left = 0
            right = 10
        if LC & (ccd == '2'):
            left = 8
            right = 20
        order_xrange_begin = np.array(polynomial_function(np.arange(np.shape(images['ccd_'+str(ccd)])[0]),*initial_order_coeffs[order])+left,dtype=int)
        order_xrange_end   = np.array(polynomial_function(np.arange(np.shape(images['ccd_'+str(ccd)])[0]),*initial_order_coeffs[order])+right,dtype=int)

        if tramline_debug:
            if ccd == '2':
                plt.plot(order_xrange_begin,np.arange(len(order_xrange_begin)),c='C0',lw=0.5)
                plt.plot(order_xrange_end,np.arange(len(order_xrange_begin)),c='C1',lw=0.5)
        
        # Save the flux from each tramlines in a row; give NaN values to regions without flux
        order_counts = np.zeros(np.shape(images['ccd_'+str(ccd)])[1]); order_counts[:] = np.nan
        for x_index, x in enumerate(initial_order_ranges[order]):
            order_counts[initial_order_ranges[order][0] + x_index] = np.sum(images['ccd_'+str(ccd)][x,order_xrange_begin[x_index]:order_xrange_end[x_index]],axis=0)
        counts_in_orders.append(order_counts)

    if tramline_debug:
        plt.xlim(2500,4200)
        plt.ylim(2000,2500)
        plt.show()
        plt.close()
        
    return(np.array(counts_in_orders))

# Extract Master Flat
print('Extracting Master Flat')
master_flat = extract_orders(
    ccd1_runs = calibration_runs['Flat_60.0'][:1],
    ccd2_runs = calibration_runs['Flat_1.0'][:1],
    ccd3_runs = calibration_runs['Flat_0.1'][:1],
    Flat = True
)

# Extract Master ThXe
print('Extracting Master ThXe')
master_thxe = extract_orders(
    ccd1_runs = calibration_runs['FibTh_180.0'][:1],
    ccd2_runs = calibration_runs['FibTh_60.0'][:1],
    ccd3_runs = calibration_runs['FibTh_15.0'][:1]
)

# Extract Master LC
print('Extracting Master LC')
master_lc = extract_orders(
    ccd1_runs = calibration_runs['SimLC'][-1:],
    ccd2_runs = calibration_runs['SimLC'][-1:],
    ccd3_runs = calibration_runs['SimLC'][-1:],
    LC = True,
    # tramline_debug = True
)

# Extract Science Objects
for science_object in list(science_runs.keys())[:1]:
    print('Extracting '+science_object)
    try:
        science = extract_orders(
            ccd1_runs = science_runs[science_object],
            ccd2_runs = science_runs[science_object],
            ccd3_runs = science_runs[science_object]
        )

        # Create a primary HDU and HDU list
        primary_hdu = fits.PrimaryHDU()
        hdul = fits.HDUList([primary_hdu])

        # Loop over your extension names and corresponding data arrays
        for ext_index, ext_name in enumerate(initial_order_coeffs):
            # Create an ImageHDU object for each extension

            # Define the columns with appropriate formats
            col1_def = fits.Column(name='wave',    format='E', array=np.arange(len(science[ext_index,:]),dtype=float))
            col2_def = fits.Column(name='science', format='E', array=science[ext_index,:])
            col3_def = fits.Column(name='flat',    format='E', array=master_flat[ext_index,:])
            col4_def = fits.Column(name='thxe',    format='E', array=master_thxe[ext_index,:])
            col5_def = fits.Column(name='lc',      format='E', array=master_lc[ext_index,:])

            print(science_object+'_'+ext_name.lower())
            hdu = fits.BinTableHDU.from_columns([col1_def, col2_def, col3_def, col4_def, col5_def], name=science_object+'_'+ext_name.lower())

            # Append the HDU to the HDU list
            hdul.append(hdu)

        # Save to a new FITS file with an extension for each order
        Path(config.working_directory+'reduced_data/'+config.date+'/minimal_spectra/'+science_object).mkdir(parents=True, exist_ok=True)    
        hdul.writeto(config.working_directory+'reduced_data/'+config.date+'/minimal_spectra/'+science_object+'/minimal_'+science_object+'.fits', overwrite=True)

    except:
        print('Failed to extract '+science_object)

Extracting Master Flat
Extracting Master ThXe
Extracting Master LC
Extracting HIP37664
HIP37664_ccd_1_order_138
HIP37664_ccd_1_order_139
HIP37664_ccd_1_order_140
HIP37664_ccd_1_order_141
HIP37664_ccd_1_order_142
HIP37664_ccd_1_order_143
HIP37664_ccd_1_order_144
HIP37664_ccd_1_order_145
HIP37664_ccd_1_order_146
HIP37664_ccd_1_order_147
HIP37664_ccd_1_order_148
HIP37664_ccd_1_order_149
HIP37664_ccd_1_order_150
HIP37664_ccd_1_order_151
HIP37664_ccd_1_order_152
HIP37664_ccd_1_order_153
HIP37664_ccd_1_order_154
HIP37664_ccd_1_order_155
HIP37664_ccd_1_order_156
HIP37664_ccd_1_order_157
HIP37664_ccd_1_order_158
HIP37664_ccd_1_order_159
HIP37664_ccd_1_order_160
HIP37664_ccd_1_order_161
HIP37664_ccd_1_order_162
HIP37664_ccd_1_order_163
HIP37664_ccd_1_order_164
HIP37664_ccd_1_order_165
HIP37664_ccd_1_order_166
HIP37664_ccd_1_order_167
HIP37664_ccd_2_order_103
HIP37664_ccd_2_order_104
HIP37664_ccd_2_order_105
HIP37664_ccd_2_order_106
HIP37664_ccd_2_order_107
HIP37664_ccd_2_order_108
HIP37664_ccd_